<a href="https://colab.research.google.com/github/AnjanaSengupta/MLE-Oct-2022/blob/main/ASG_protein_pfam_protbert_xgboost_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

import torch
torch.manual_seed(42)
from google.colab import drive
drive.mount('/content/drive')

#data_path = 'drive/MyDrive/MLE/protein_data/'
data_path = '/content/drive/MyDrive/random_split/'
GPU = True

Mounted at /content/drive


In [ ]:
df = pd.read_parquet(data_path + 'train_protbert_embeddings.parquet')
df.head(1)

,family_id,sequence_name,family_accession,aligned_sequence,sequence,numpy_embeddings
8,NBD_C,R5HG95_9SPIR/261-424,PF17042.5,FVVSGSATQITANQIEKLEEADEF.ENL.LVI......N.LDMKTV...,FVVSGSATQITANQIEKLEEADEFENLLVINLDMKTVLAGVSDEIV...,"[-19.729471, -19.531374, -19.430761, -21.61242..."


In [ ]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn.cluster
from sklearn.metrics import roc_auc_score


def create_xgboost(X, y, **model_kwargs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    xgboost_gpu_kwargs = {"gpu_id": 0, "tree_method": 'gpu_hist'} if GPU else {}
    model = xgboost.XGBClassifier(verbosity=1, **xgboost_gpu_kwargs, **model_kwargs)

    # Fit XGBoost
    # TODO: play with eval_metric?
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)
    # Predict the test set labels
    y_preb_probs = model.predict_proba(X_test)
    y_pred = model.predict(X_test)
    # Analyze results
    print(classification_report(y_test, y_pred))

    #print(roc_auc_score(y_test, y_preb_probs, average="weighted", multi_class="ovr"))
    return model

X = pd.DataFrame(df['numpy_embeddings'].to_list())
y = df['family_id']

In [ ]:
xgboost = create_xgboost(X, y)

[0]	validation_0-merror:0.79547
[1]	validation_0-merror:0.75374
[2]	validation_0-merror:0.726425
[3]	validation_0-merror:0.704945
[4]	validation_0-merror:0.687437
[5]	validation_0-merror:0.671327
[6]	validation_0-merror:0.655946
[7]	validation_0-merror:0.643375
[8]	validation_0-merror:0.632419
[9]	validation_0-merror:0.62218
[10]	validation_0-merror:0.611748
[11]	validation_0-merror:0.601782
[12]	validation_0-merror:0.593283
[13]	validation_0-merror:0.584887
[14]	validation_0-merror:0.577003
[15]	validation_0-merror:0.56962
[16]	validation_0-merror:0.562589
[17]	validation_0-merror:0.555649
[18]	validation_0-merror:0.548743
[19]	validation_0-merror:0.542122
[20]	validation_0-merror:0.535524
[21]	validation_0-merror:0.529881
[22]	validation_0-merror:0.523521
[23]	validation_0-merror:0.518265
[24]	validation_0-merror:0.512031
[25]	validation_0-merror:0.507139
[26]	validation_0-merror:0.502327
[27]	validation_0-merror:0.497241
[28]	validation_0-merror:0.492452
[29]	validation_0-merror:0.4